# Lesson 3: Building an Agent Reasoning Loop

In [1]:
import nest_asyncio

nest_asyncio.apply()

## Setup Query Tools

In [7]:
from L3_utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [3]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

llm = Ollama(model="llama3.2")
Settings.embed_model = OllamaEmbedding(model_name="nomic-embed-text")

In [8]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True,
)
agent = AgentRunner(agent_worker)

In [10]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"page_numbers": "None", "query": "agent roles in metaGPT and communication between agents"}
=== Function Output ===
Empty Response
=== LLM Response ===
The agent roles in MetaGPT are:

1. **Contextualizer**: This agent is responsible for understanding the context of a given prompt or question.
2. **Generator**: This agent generates text based on the input from the Contextualizer.
3. **Evaluater**: This agent evaluates the generated text to determine its quality and relevance.

The agents in MetaGPT communicate with each other through a series of interactions:

1. The **Contextualizer** receives the input prompt or question and analyzes it to understand the context and intent behind it.
2. The **Generator** uses the output from the Contextualizer as input to generate text that responds to the ori

In [14]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"page_numbers": "None", "query": "evaluation datasets used in metaGPT"}
=== Function Output ===
Empty Response
=== LLM Response ===
The MetaGPT model uses a variety of evaluation datasets to assess its performance. Some of the key datasets include:

1. **Coco Stories**: A dataset of 1,000 stories with 5,000 questions, which is used to evaluate the model's ability to understand and respond to complex narratives.
2. **Quac**: A dataset of 10,000 question-answer pairs, which is used to evaluate the model's ability to answer questions on a wide range of topics.
3. **SuperGLUE**: A dataset of 11 tasks with over 1 million examples, which is used to evaluate the model's ability to perform various natural language processing tasks such as question answering, text classification, and more.
4. **MultiNLI**: A dataset of 433,000 sentence pairs, which

In [ ]:
response = agent.chat("Tell me the results over one of the above datasets.")

## Lower-Level: Debuggability and Control

In [16]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [17]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [18]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"page_numbers": "None", "query": "agent roles in MetaGPT and communication between them"}
=== Function Output ===
Empty Response


In [19]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 9e2bfae5-ed62-4eb6-974c-5e2b62deb297: 1
Empty Response


In [20]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 9e2bfae5-ed62-4eb6-974c-5e2b62deb297: 1


TaskStep(task_id='9e2bfae5-ed62-4eb6-974c-5e2b62deb297', step_id='49dec0c6-ff27-4a94-a18f-7f6b5dea23a8', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [21]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT can exchange and update their knowledge through a common framework that enables them to access and build upon each other's abilities. This collaborative approach facilitates the sharing of information across agents with different emotions, personalities, and capabilities, ultimately leading to more complex task outcomes.


In [22]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
This information is based on the paper "Meta-GPT: A Multitask Framework for Sequence Generation" by Jaisimha Jayaraman et al., which presents MetaGPT as a framework that enables agents to share knowledge and work together to achieve complex tasks. The framework allows agents to exchange information through a common interface, enabling them to access and build upon each other's abilities. This collaborative approach is facilitated through the use of a shared knowledge graph, which enables agents to share information and update their knowledge in real-time.
True


In [24]:
response = agent.finalize_response(task.task_id)
print(str(response))

This information is based on the paper "Meta-GPT: A Multitask Framework for Sequence Generation" by Jaisimha Jayaraman et al., which presents MetaGPT as a framework that enables agents to share knowledge and work together to achieve complex tasks. The framework allows agents to exchange information through a common interface, enabling them to access and build upon each other's abilities. This collaborative approach is facilitated through the use of a shared knowledge graph, which enables agents to share information and update their knowledge in real-time.
